<a href="https://colab.research.google.com/github/brianpyai/chatbot/blob/main/gemma_playgrounds_GGUF_4Bit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install gradio==4.22.0 transformers

  Using cached gradio-4.22.0-py3-none-any.whl (17.1 MB)
  Using cached aiofiles-23.2.1-py3-none-any.whl (15 kB)
  Using cached fastapi-0.110.2-py3-none-any.whl (91 kB)
  Using cached ffmpy-0.3.2.tar.gz (5.5 kB)
  Preparing metadata (setup.py) ... done
  Using cached gradio_client-0.13.0-py3-none-any.whl (311 kB)
  Using cached httpx-0.27.0-py3-none-any.whl (75 kB)
  Using cached orjson-3.10.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (141 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)
  Using cached python_multipart-0.0.9-py3-none-any.whl (22 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━

In [ ]:

#@title Built llama-cpp-python with BLAS or CUDA automatically, wait for few minutes .
import os,torch


if torch.cuda.is_available ():
    cmd='CMAKE_ARGS="%s" pip install llama-cpp-python==0.2.62' % "-DLLAMA_CUBLAS=on"
else:
    cmd='CMAKE_ARGS="%s" pip install llama-cpp-python==0.2.62' % "-DLLAMA_BLAS=ON -DLLAMA_BLAS_VENDOR=OpenBLAS"

os.system (cmd)

0

In [ ]:




from llama_cpp import Llama,LlamaRAMCache,LlamaDiskCache,LlamaTokenizer,LlamaState
from llama_cpp.llama_speculative import LlamaPromptLookupDecoding
from llama_cpp.llama_chat_format import Llava15ChatHandler
import os,torch,json,shlex
from pathlib import Path
from PIL import Image
from huggingface_hub import hf_hub_download

try:
  import google.colab
  IN_COLAB = True
except :IN_COLAB=False
Gbase="./generate/"
cache_dir="./hf/"

import torch
from psutil import cpu_count
import platform
if torch.cuda.is_available():
    n_gpu_layers=-1
    n_threads=cpu_count()
else :
    n_gpu_layers=0
    n_threads=cpu_count()

if platform.machine()=='a0arch64' and not IN_COLAB:n_threads=4


description ="""<div style="font-family: Arial, sans-serif; padding: 20px;">

This showcases the use of llama-cpp-python to load different versions of 4-bit and 8-bit models, as well as a simple chat interface created by Gradio. It can run on small computers or even mobile devices and produce satisfactory results. You are free to modify this code. I will continue to update these tools, first unlocking more possibilities to allow this tool to handle most common content, and then combining all of these possibilities with automation elements. Ultimately, it will evolve into a powerful tool that solves real-world problems effectively.

這裡展示了使用llama-cpp-python來加載各種4位和8位模型的不同版本，以及由Gradio創建的簡單聊天界面。它可以在小型電腦甚至手機上運行，並能產生令人滿意的結果。您可以自由修改這段代碼。我將繼續更新這些工具，首先解鎖更多可能性，使此工具能夠處理大多數常見內容，然後將所有這些可能性與自動化元素結合在一起。最終，它將發展成一個有效解決現實問題的工具。
  <h2>🔗My social media links❤️</h2>

Follow <a href="https://www.facebook.com/braiml"  ytarget="_blank">🐍Brian's Page </a> if you want I share more tools .<br>
Follow<a href="https://www.facebook.com/charactersAI" target="_blank">❤️Characters AI</a>
 if you want more  videos .
<br>
  <a href="https://www.facebook.com/brian.pyai" target="_blank"> 📘facebook.com/brian.pyai</a>
 <br>
  <a href="https://www.facebook.com/braiml" target="_blank">🐍Brian's Page </a>
<br>

  <a href="https://www.facebook.com/lovelyai999" target="_blank">🥰AI Hot Shorts </a>



</div>

"""
import base64
import io
def image_to_base64_data_uri(file_path):
    i=Image.open(file_path)
    i.resize((256,256))
    img_byte_arr = io.BytesIO()
    i.save(img_byte_arr, format='PNG')
    base64_data = base64.b64encode(img_byte_arr.getvalue()).decode('utf-8')
    return f"data:image/png;base64,{base64_data}"

modelsPath="./"

modelsPaths= ["mradermacher/llava-v1.6-mistral-7b-GGUF/llava-v1.6-mistral-7b.Q4_K_M.gguf","SanctumAI/Phi-3-mini-4k-instruct-GGUF/phi-3-mini-4k-instruct.Q4_K_M.gguf","MaziyarPanahi/WizardLM-2-7B-GGUF/WizardLM-2-7B.Q4_K_M.gguf","SanctumAI/Phi-3-mini-4k-instruct-GGUF/phi-3-mini-4k-instruct.Q8_0.gguf","MaziyarPanahi/WizardLM-2-7B-GGUF/WizardLM-2-7B.Q8_0.gguf","FaradayDotDev/llama-3-8b-Instruct-GGUF/llama-3-8b-Instruct.Q4_K_M.gguf","seyf1elislam/llama-3-neural-chat-v1-8b-GGUF/llama-3-neural-chat-v1-8b.Q4_K_M.gguf","seyf1elislam/llama-3-neural-chat-v1-8b-GGUF/llama-3-neural-chat-v1-8b.Q8_0.gguf","Quant-Cartel/Llama-3-8B-Instruct-DADA-iMat-GGUF/Llama-3-8B-Instruct-DADA-iMat-IQ4_XS.gguf","Quant-Cartel/Llama-3-8B-Instruct-DADA-iMat-GGUF/Llama-3-8B-Instruct-DADA-iMat-Q8_0.gguf","3thn/dolphin-2.9-llama3-8b-GGUF/dolphin-2.9-llama3-8b.Q4_K_M.gguf","3thn/dolphin-2.9-llama3-8b-GGUF/dolphin-2.9-llama3-8b.Q8_0.gguf","PawanKrd/Meta-Llama-3-70B-Instruct-GGUF/llama-3-70b-instruct.Q3_K_M.gguf","twodgirl/zephyr-beta-wizardLM-2-merge-7B-Q6_K-GGUF/zephyr-beta-wizardlm-2-merge-7b.Q6_K.gguf","MaziyarPanahi/WizardLM-2-8x22B-GGUF/WizardLM-2-8x22B.IQ1_M.gguf","TheBloke/dolphin-2.7-mixtral-8x7b-GGUF/dolphin-2.7-mixtral-8x7b.Q2_K.gguf","TheBloke/dolphin-2.7-mixtral-8x7b-GGUF/dolphin-2.7-mixtral-8x7b.Q4_K_M.gguf","mradermacher/Starling-LM-alpha-8x7B-MoE-GGUF/Starling-LM-alpha-8x7B-MoE.Q4_K_M.gguf","MaziyarPanahi/Qwen1.5-8x7b-v0.1-GGUF/Qwen1.5-8x7b-v0.1.Q4_K_M.gguf","TheBloke/firefly-mixtral-8x7b-GGUF/firefly-mixtral-8x7b.Q4_K_M.gguf","TheBloke/openbuddy-mixtral-8x7b-v15.1-GGUF/openbuddy-mixtral-8x7b-v15.1.Q4_K_M.gguf","Quant-Cartel/Cerebrum-1.0-8x7b-iMat-GGUF/Cerebrum-1.0-8x7b-iMat-Q4_K_M.gguf","MaziyarPanahi/Experiment26-7B-GGUF/Experiment26-7B.Q4_K_M.gguf","dagbs/dolphin-2.8-experiment26-7b-preview-GGUF/dolphin-2.8-experiment26-7b.Q4_K_M.gguf","ggml-org/gemma-1.1-2b-it-Q4_K_M-GGUF/gemma-1.1-2b-it.Q4_K_M.gguf","chenhunghan/TAIDE-LX-7B-Chat-GGUF/taide-lx-7b-chat.q4_k_m.gguf","chenhunghan/TAIDE-LX-7B-Chat-GGUF/taide-lx-7b-chat.q8_0.gguf","ggml-org/gemma-1.1-7b-it-Q4_K_M-GGUF/gemma-1.1-7b-it.Q4_K_M.gguf","ggml-org/gemma-1.1-2b-it-Q8_0-GGUF/gemma-1.1-2b-it.Q8_0.gguf","ggml-org/gemma-1.1-7b-it-Q8_0-GGUF/gemma-1.1-7b-it.Q8_0.gguf","pi-null-mezon/openchat-3.5-0106-gemma-GGUF/ggml-model-Q8_0.gguf","pi-null-mezon/openchat-3.5-0106-gemma-GGUF/ggml-model-Q4_K_M.gguf","mlabonne/Gemmalpaca-2B-GGUF/gemmalpaca-2b.Q4_K_M.gguf","LoneStriker/gemma-2b-GGUF/gemma-2b-Q4_K_M.gguf","LoneStriker/gemma-2b-it-GGUF/gemma-2b-it-Q4_K_M.gguf","LoneStriker/gemma-7b-GGUF/gemma-7b-Q4_K_M.gguf","LoneStriker/gemma-7b-it-GGUF/gemma-7b-it-Q4_K_M.gguf","mlabonne/Gemmalpaca-2B-GGUF/gemmalpaca-2b.Q4_K_M.gguf","LoneStriker/OrcaGemma-2B-GGUF/OrcaGemma-2B-Q4_K_M.gguf","rombodawg/EveryoneLLM-7b-Gemma-Base-GGUF/EveryoneLLM-7b-Gemma-Base-q6_k.gguf","LoneStriker/openbuddy-gemma-7b-v19.1-4k-GGUF/openbuddy-gemma-7b-v19.1-4k-Q4_K_M.gguf","LoneStriker/Gemmalpaca-7B-GGUF/Gemmalpaca-7B-Q4_K_M.gguf", "LoneStriker/zephyr-7b-gemma-v0.1-GGUF/zephyr-7b-gemma-v0.1-Q4_K_M.gguf" ,"Lewdiculous/firefly-gemma-7b-GGUF-IQ-Imatrix/firefly-gemma-7b-Q4_K_S-imatrix.gguf"]



model_id="SanctumAI/Phi-3-mini-4k-instruct-GGUF/phi-3-mini-4k-instruct.Q8_0.gguf" #@param    ["mradermacher/llava-v1.6-mistral-7b-GGUF/llava-v1.6-mistral-7b.Q4_K_M.gguf","SanctumAI/Phi-3-mini-4k-instruct-GGUF/phi-3-mini-4k-instruct.Q4_K_M.gguf","MaziyarPanahi/WizardLM-2-7B-GGUF/WizardLM-2-7B.Q4_K_M.gguf","SanctumAI/Phi-3-mini-4k-instruct-GGUF/phi-3-mini-4k-instruct.Q8_0.gguf","MaziyarPanahi/WizardLM-2-7B-GGUF/WizardLM-2-7B.Q8_0.gguf","FaradayDotDev/llama-3-8b-Instruct-GGUF/llama-3-8b-Instruct.Q4_K_M.gguf","seyf1elislam/llama-3-neural-chat-v1-8b-GGUF/llama-3-neural-chat-v1-8b.Q4_K_M.gguf","seyf1elislam/llama-3-neural-chat-v1-8b-GGUF/llama-3-neural-chat-v1-8b.Q8_0.gguf","Quant-Cartel/Llama-3-8B-Instruct-DADA-iMat-GGUF/Llama-3-8B-Instruct-DADA-iMat-IQ4_XS.gguf","Quant-Cartel/Llama-3-8B-Instruct-DADA-iMat-GGUF/Llama-3-8B-Instruct-DADA-iMat-Q8_0.gguf","3thn/dolphin-2.9-llama3-8b-GGUF/dolphin-2.9-llama3-8b.Q4_K_M.gguf","3thn/dolphin-2.9-llama3-8b-GGUF/dolphin-2.9-llama3-8b.Q8_0.gguf","PawanKrd/Meta-Llama-3-70B-Instruct-GGUF/llama-3-70b-instruct.Q3_K_M.gguf","twodgirl/zephyr-beta-wizardLM-2-merge-7B-Q6_K-GGUF/zephyr-beta-wizardlm-2-merge-7b.Q6_K.gguf","MaziyarPanahi/WizardLM-2-8x22B-GGUF/WizardLM-2-8x22B.IQ1_M.gguf","TheBloke/dolphin-2.7-mixtral-8x7b-GGUF/dolphin-2.7-mixtral-8x7b.Q2_K.gguf","TheBloke/dolphin-2.7-mixtral-8x7b-GGUF/dolphin-2.7-mixtral-8x7b.Q4_K_M.gguf","mradermacher/Starling-LM-alpha-8x7B-MoE-GGUF/Starling-LM-alpha-8x7B-MoE.Q4_K_M.gguf","MaziyarPanahi/Qwen1.5-8x7b-v0.1-GGUF/Qwen1.5-8x7b-v0.1.Q4_K_M.gguf","TheBloke/firefly-mixtral-8x7b-GGUF/firefly-mixtral-8x7b.Q4_K_M.gguf","TheBloke/openbuddy-mixtral-8x7b-v15.1-GGUF/openbuddy-mixtral-8x7b-v15.1.Q4_K_M.gguf","Quant-Cartel/Cerebrum-1.0-8x7b-iMat-GGUF/Cerebrum-1.0-8x7b-iMat-Q4_K_M.gguf","MaziyarPanahi/Experiment26-7B-GGUF/Experiment26-7B.Q4_K_M.gguf","dagbs/dolphin-2.8-experiment26-7b-preview-GGUF/dolphin-2.8-experiment26-7b.Q4_K_M.gguf","ggml-org/gemma-1.1-2b-it-Q4_K_M-GGUF/gemma-1.1-2b-it.Q4_K_M.gguf","chenhunghan/TAIDE-LX-7B-Chat-GGUF/taide-lx-7b-chat.q4_k_m.gguf","chenhunghan/TAIDE-LX-7B-Chat-GGUF/taide-lx-7b-chat.q8_0.gguf","ggml-org/gemma-1.1-7b-it-Q4_K_M-GGUF/gemma-1.1-7b-it.Q4_K_M.gguf","ggml-org/gemma-1.1-2b-it-Q8_0-GGUF/gemma-1.1-2b-it.Q8_0.gguf","ggml-org/gemma-1.1-7b-it-Q8_0-GGUF/gemma-1.1-7b-it.Q8_0.gguf","pi-null-mezon/openchat-3.5-0106-gemma-GGUF/ggml-model-Q8_0.gguf","pi-null-mezon/openchat-3.5-0106-gemma-GGUF/ggml-model-Q4_K_M.gguf","mlabonne/Gemmalpaca-2B-GGUF/gemmalpaca-2b.Q4_K_M.gguf","LoneStriker/gemma-2b-GGUF/gemma-2b-Q4_K_M.gguf","LoneStriker/gemma-2b-it-GGUF/gemma-2b-it-Q4_K_M.gguf","LoneStriker/gemma-7b-GGUF/gemma-7b-Q4_K_M.gguf","LoneStriker/gemma-7b-it-GGUF/gemma-7b-it-Q4_K_M.gguf","mlabonne/Gemmalpaca-2B-GGUF/gemmalpaca-2b.Q4_K_M.gguf","LoneStriker/OrcaGemma-2B-GGUF/OrcaGemma-2B-Q4_K_M.gguf","rombodawg/EveryoneLLM-7b-Gemma-Base-GGUF/EveryoneLLM-7b-Gemma-Base-q6_k.gguf","LoneStriker/openbuddy-gemma-7b-v19.1-4k-GGUF/openbuddy-gemma-7b-v19.1-4k-Q4_K_M.gguf","LoneStriker/Gemmalpaca-7B-GGUF/Gemmalpaca-7B-Q4_K_M.gguf", "LoneStriker/zephyr-7b-gemma-v0.1-GGUF/zephyr-7b-gemma-v0.1-Q4_K_M.gguf" ,"Lewdiculous/firefly-gemma-7b-GGUF-IQ-Imatrix/firefly-gemma-7b-Q4_K_S-imatrix.gguf"]
def selectPath(paths=modelsPaths):
    #return "gemma-openchat7v-model-Q4_K_M.gguf"
    ls=paths
    #c='termux-dialog radio -v "%s" -t "Select model"' % ",".join(ls)
    #v=eval(os.popen(c).read())["text"]
    for i,t in enumerate(ls):print ("%s) %s" % (i ,t) )
    print ("input the number :")
    v=ls[int(input())]

    return v
if not IN_COLAB:
    model_id=selectPath()
def downHgFile(url,targetDir=modelsPath):
    fileName=Path (model_id ).name
    repo =model_id[:-len(fileName)-1]
    print(fileName,repo)
    fileExists=os.path.exists(os.path.join(targetDir,fileName))

    print(repo,fileName,fileExists )
    if not fileExists:
        print ("Downloading file :")
        hf_hub_download(repo ,filename=fileName,local_dir=targetDir,local_dir_use_symlinks=False)
max_tokens=4096 # @param {type:"integer",min:10, max:8192}
n_ctx=4096 # @param {type:"integer",min:10, max:8192}
downHgFile(model_id )
#hf_hub_download("lovelyai999/temp" ,filename="mistral_7b_mmproj-v1_5_Q4_1.gguf",local_dir="./",local_dir_use_symlinks=False)
modelPath=os.path.join(modelsPath,Path(model_id).name)
if ("8x" in model_id or "70b" in model_id) and torch.cuda.is_available():
    n_gpu_layers=16
    if  "70b" in model_id:
        n_gpu_layers=24
    n_threads=4
if model_id=="mradermacher/llava-v1.6-mistral-7b-GGUF/llava-v1.6-mistral-7b.Q4_K_M.gguf":
    if not os.path.exists("mistral_7b_mmproj-v1_5_Q4_1.gguf"):hf_hub_download("lovelyai999/temp" ,filename="mistral_7b_mmproj-v1_5_Q4_1.gguf",local_dir="./",local_dir_use_symlinks=False)
    chat_handler = Llava15ChatHandler(clip_model_path="mistral_7b_mmproj-v1_5_Q4_1.gguf",verbose=True)
    model = Llama(modelPath,n_gpu_layers=n_gpu_layers,n_threads=n_threads,max_tokens=4096,logits_all=True,n_ctx=n_ctx,chat_handler=chat_handler)
else:
    model = Llama(modelPath,n_gpu_layers=n_gpu_layers,n_threads=n_threads,max_tokens=4096,logits_all=True,n_ctx=n_ctx)
#draft_model=LlamaPromptLookupDecoding(num_pred_tokens=10) )

tokenizer = model.tokenize




import gradio as gr
import torch

from threading import Thread




partial_message = ""

def predict(message, history,top_k=300, top_p=0.95, temp=0.85, repeat_penalty=1.1,max_tokens=max_tokens):
    global partial_message,model_id
    try :messageT=message["text"]
    except :messageT=message
    global model_id
    if "phi-3-" in model_id:
        prompt=f"<|user|>{messageT}<|end|><|assistant|>"
    else:
        prompt=f"""### System:
You are a professional private assistant.
### User:
{messageT}
###  Response:

"""
    prompt_=f"""
###Below is an instruction that describes my question or task.
Write a response that appropriately completes the request:
{messageT}
### Response :

"""

    print (message)
    #model_inputs = tokenizer(prompt.encode("utf-8"))
    #generate_kwargs = dict(top_k=top_k, top_p=top_p, temp=temp, repeat_penalty=repeat_penalty)
    stop= ["<|end|>" ,"<|end_of_text|>"  ]
    if model_id in ["FaradayDotDev/llama-3-8b-Instruct-GGUF/llama-3-8b-Instruct.Q4_K_M.gguf","l3utterfly/llama-3-8b-Instruct-gguf/llama-3-8b-Instruct-Q4_K.gguf","l3utterfly/llama-3-8b-Instruct-gguf/llama-3-8b-Instruct-Q8_0.gguf","PawanKrd/Meta-Llama-3-70B-Instruct-GGUFFaradayDotDev/llama-3-8b-Instruct-GGUF/llama-3-70b-instruct.Q2_K.gguf","PawanKrd/Meta-Llama-3-70B-Instruct-GGUF/llama-3-70b-instruct.Q3_K_M.gguf"]:
        stop=["### Below",".\n\n","assistant\n\n","!\n\n","<|end|>" ,"<|end_of_text|>" ]
    generate_kwargs=dict (suffix=None, max_tokens=max_tokens, temperature=temp, top_p=top_p, min_p=0.05, typical_p=1.0, logprobs=None, echo=False, stop=stop, frequency_penalty=0,presence_penalty=0.0, repeat_penalty=1.1, top_k=top_k, stream=True , seed=None, tfs_z=1.0, mirostat_mode=0, mirostat_tau=5.0, mirostat_eta=0.1, model=None, stopping_criteria=None, logits_processor=None, grammar=None, logit_bias=None)


    partial_message = ""
    if isinstance(message,dict) and message["files"] and message["files"][0]["path"] and message["files"][0]['mime_type'].startswith("image"):
        messages = [
        {"role": "system", "content": "You are an assistant who perfectly describes images."},
        {
            "role": "user",
            "content": [
                {"type": "image_url", "image_url": {"url":message["files"][0]["path"]}},
                {"type" : "text", "text": messageT}
            ]
        }]
        outputs =model.create_completion(json.dumps (messages),**generate_kwargs)
    else:
        outputs =model.create_completion(prompt ,**generate_kwargs)


    for chunk in outputs:
        #print (chunk )
        content = chunk["choices"][0]["text"]
        #print(content,repr(content ))
        if content:
            partial_message+=content
            yield partial_message
    """
    for new_token in model.generate(model_inputs,**generate_kwargs):
            #print (new_token)
            if new_token in (213,7221,1):break
            s=model.detokenize([new_token])
            #print (s)
            if isinstance(s,bytes):
                try:partial_message += s.decode("utf-8")
                except :break
            yield partial_message
        """

chatbot = gr.Chatbot(likeable=True,show_copy_button=True)
textbox = gr.Textbox (show_copy_button=True)

user="brian" #@param {type:"string"}
password="pwd" #@param {type:"string"}
auth=(user,password)

gr.ChatInterface(predict,chatbot=chatbot,stop_btn="Stop" ,retry_btn="Retry",concurrency_limit=1,description=description,
additional_inputs=[gr.Number(value=100,minimum =0,maximum =1000,precision=0,show_label=True ,label="Topic K"),
gr.Number(value=0.85,minimum =0,maximum =1,precision=3,show_label=True ,label="Topic P"),
gr.Number(value=0.85,minimum =0,maximum =1,precision=3,show_label=True ,label="Temperature"),gr.Number(value=1.1,minimum =1,maximum =3,precision=3,show_label=True ,label="Repeat penalty") ,gr.Number(value=4096,minimum =0,maximum =8192,precision=0,show_label=True ,label="max_tokens") ],multimodal=True ).launch(debug=True,share=True ,inline=False,auth=auth )

llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from ./dolphin-2.9-llama3-8b.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 128258
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                          llama.block_count u32              = 32
llama_model_loader: - kv   6:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   7:                 llama.rope.dimension_count u32   

dolphin-2.9-llama3-8b.Q4_K_M.gguf 3thn/dolphin-2.9-llama3-8b-GGUF
3thn/dolphin-2.9-llama3-8b-GGUF dolphin-2.9-llama3-8b.Q4_K_M.gguf True


llama_model_loader: - kv  17:                      tokenizer.ggml.merges arr[str,280147]  = ["Ġ Ġ", "Ġ ĠĠĠ", "ĠĠ ĠĠ", "...
llama_model_loader: - kv  18:                tokenizer.ggml.bos_token_id u32              = 128000
llama_model_loader: - kv  19:                tokenizer.ggml.eos_token_id u32              = 128256
llama_model_loader: - kv  20:            tokenizer.ggml.padding_token_id u32              = 128001
llama_model_loader: - kv  21:                    tokenizer.chat_template str              = {% set loop_messages = messages %}{% ...
llama_model_loader: - kv  22:               general.quantization_version u32              = 2
llama_model_loader: - type  f32:   65 tensors
llama_model_loader: - type q4_K:  193 tensors
llama_model_loader: - type q6_K:   33 tensors
llm_load_vocab: special tokens definition check successful ( 258/128258 ).
llm_load_print_meta: format           = GGUF V3 (latest)
llm_load_print_meta: arch             = llama
llm_load_print_meta: vocab type      

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://150c6fda8883d28d16.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
{'text': "Using Javascript and CSS to create a beautiful lunar calendar, accurately displaying today's date, and all codes within the same HTML.", 'files': []}
Creating 'Creating'
 an ' an'
 interactive ' interactive'
 and ' and'
 visually ' visually'
 appealing ' appealing'
 Lunar ' Lunar'
 Calendar ' Calendar'
 using ' using'
 JavaScript ' JavaScript'
 ( ' ('
JS 'JS'
) ')'
 and ' and'
 Casc ' Casc'
ading 'ading'
 Style ' Style'
 Sheets ' Sheets'
 ( ' ('
CSS 'CSS'
) ')'
 can ' can'
 be ' be'
 quite ' quite'
 engaging ' engaging'
. '.'
 To ' To'
 assist ' assist'
 you ' you'
 with ' with'
 this ' this'
 task ' task'
, ','
 we

Llama.generate: prefix-match hit


, ','
 I ' I'
 can ' can'
 help ' help'
 you ' you'
 with ' with'
 that ' that'
.

 '.\n\n'
Here 'Here'
 is ' is'
 an ' an'
 example ' example'
 of ' of'
 how ' how'
 you ' you'
 could ' could'
 structure ' structure'
 your ' your'
 code ' code'
 in ' in'
 HTML ' HTML'
, ','
 JavaScript ' JavaScript'
 and ' and'
 CSS ' CSS'
:

 ':\n\n'
HTML 'HTML'
:
 ':\n'
``` '```'
html 'html'

 '\n'
<! '<!'
DOCTYPE 'DOCTYPE'
 html ' html'
>
 '>\n'
<html '<html'
>
 '>\n'
<head '<head'
>
 '>\n'
	 '\t'
<link '<link'
 rel ' rel'
=" '="'
stylesheet 'stylesheet'
" '"'
 type ' type'
=" '="'
text 'text'
/css '/css'
" '"'
 href ' href'
=" '="'
style 'style'
.css '.css'
">
 '">\n'
</ '</'
head 'head'
>

 '>\n\n'
<body '<body'
 onload ' onload'
=" '="'
load 'load'
Data 'Data'
()">
 '()">\n'
	 '\t'
<h '<h'
1 '1'
>L '>L'
unar 'unar'
 Calendar ' Calendar'
</ '</'
h 'h'
1 '1'
>
 '>\n'
	 '\t'
<p '<p'
 id ' id'
=" '="'
date 'date'
"></ '"></'
p 'p'
>
 '>\n'
	
 '\t\n'
	 '\t'
<script '<script'
 src ' src'
=" '="'
scrip


llama_print_timings:        load time =   32248.85 ms
llama_print_timings:      sample time =     584.35 ms /   245 runs   (    2.39 ms per token,   419.27 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   50575.26 ms /   245 runs   (  206.43 ms per token,     4.84 tokens per second)
llama_print_timings:       total time =   54856.46 ms /   246 tokens


 ''
{'text': "Using Javascript and CSS to create a beautiful lunar calendar, accurately displaying today's date, and all codes within the same HTML.", 'files': []}
Creating 'Creating'


Llama.generate: prefix-match hit


 a ' a'
 lunar ' lunar'
 calendar ' calendar'
 using ' using'
 JavaScript ' JavaScript'
 ( ' ('
JS 'JS'
) ')'
 and ' and'
 Casc ' Casc'
ading 'ading'
 Style ' Style'
 Sheets ' Sheets'
 ( ' ('
CSS 'CSS'
), '),'
 while ' while'
 also ' also'
 writing ' writing'
 all ' all'
 of ' of'
 the ' the'
 code ' code'
 inside ' inside'
 the ' the'
 same ' same'
 HTML ' HTML'
 file ' file'
 is ' is'
 certainly ' certainly'
 possible ' possible'
. '.'
 Here ' Here'
 is ' is'
 an ' an'
 example ' example'
 of ' of'
 how ' how'
 you ' you'
 can ' can'
 begin ' begin'
 to ' to'
 structure ' structure'
 your ' your'
 HTML ' HTML'
 document ' document'
 with ' with'
 JS ' JS'
 and ' and'
 CSS ' CSS'
 included ' included'
:

 ':\n\n'
``` '```'
HTML 'HTML'

 '\n'
<! '<!'
DOCTYPE 'DOCTYPE'
 html ' html'
>
 '>\n'
<html '<html'
>
 '>\n'
    '   '
 < ' <'
head 'head'
>
 '>\n'
        '       '
 <!-- ' <!--'
 CSS ' CSS'
 Styles ' Styles'
 -->
 ' -->\n'
        '       '
 < ' <'
style 'style'
>
 '>\n'
          